In [75]:
#!pip install pandas
import pandas as pd
import xgboost as xgb
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from collections import Counter
from tqdm import tqdm
from sklearn.cluster import KMeans

Разберемся с данными. В transactions_train и transactions_test находятся транзакции пользователей. Пользователи представлены анонимизированными client_id. Каждая транзакция представляет из себя дату, группу товара и потраченную сумму, соответственно trans_date, small_group, amount_rur. Дата и потраченная сумма также анонимизированы. В train_target находятся пары client_id и bins, где bins является 0, 1, 2 или 3, соответствующей анонимизированной возрастной группой, которую нам и требуется предсказать.

In [39]:
transactions_train = pd.read_csv('/home/alex/Рабочий стол/data_nti/data/transactions_train.csv')

In [101]:
transactions_train.head()

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341


In [40]:
train_target = pd.read_csv('/home/alex/Рабочий стол/data_nti/data/train_target.csv')

In [100]:
train_target.head()

,client_id,bins
0,24662,2
1,1046,0
2,34089,2
3,34848,1
4,47076,3


Посчитаем простые агрегационные признаки по всем группам и по каждой в отдельности. Сумму и количество нормализуем, взяв не абсолютное значение, а доли от суммы и количества по всем группам.

In [41]:
agg_features = transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max', 'count', 'median']).reset_index()

In [102]:
agg_features.head()

,client_id,sum,mean,std,min,max,count,median
0,4,28404.121,39.450168,73.511624,0.043,1341.802,720,22.725
1,6,15720.739,21.535259,26.200397,0.045,315.781,730,13.898
2,7,53630.036,69.379089,253.261383,0.043,4505.971,773,27.477
3,10,34419.365,48.752642,63.191701,0.045,654.893,706,32.057
4,11,26789.404,32.991877,107.395139,0.388,2105.058,812,11.066


In [103]:
train = pd.merge(train_target,agg_features,on='client_id')

In [104]:
counter_df_train = transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
cat_counts_train = counter_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_counts_train = cat_counts_train.fillna(0)
cat_counts_train.columns = ['small_group_'+str(i) for i in cat_counts_train.columns]
train = pd.merge(train,cat_counts_train.reset_index(),on='client_id')
for name in cat_counts_train.columns.to_list():    # Нормализуем, взяв долю
    train[name] /= train['count']

In [105]:
summer_df_train = transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
cat_sums_train = summer_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_sums_train = cat_sums_train.fillna(0)
cat_sums_train.columns = ['small_group_sum_'+str(i) for i in cat_sums_train.columns]
train = pd.merge(train,cat_sums_train.reset_index(),on='client_id')
for name in cat_sums_train.columns.to_list():    # Нормализуем, взяв долю
    train[name] /= train['sum']

In [106]:
mean_df_train = transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
mean_df_train = mean_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_mean_train = mean_df_train.fillna(0)
cat_mean_train.columns = ['small_group_mean_'+str(i) for i in cat_mean_train.columns]
train = pd.merge(train,cat_mean_train.reset_index(),on='client_id')

In [107]:
std_df_train = transactions_train.groupby(['client_id','small_group'])['amount_rur'].std()
std_df_train = std_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_std_train = std_df_train.fillna(0)
cat_std_train.columns = ['small_group_std_'+str(i) for i in cat_std_train.columns]
train = pd.merge(train,cat_std_train.reset_index(),on='client_id')

In [108]:
max_df_train = transactions_train.groupby(['client_id','small_group'])['amount_rur'].max()
max_df_train = max_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_max_train = max_df_train.fillna(0)
cat_max_train.columns=['small_group_max_'+str(i) for i in cat_max_train.columns]
train = pd.merge(train,cat_max_train.reset_index(),on='client_id')

In [109]:
min_df_train = transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
min_df_train = min_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_min_train = min_df_train.fillna(0)
cat_min_train.columns=['small_group_min_'+str(i) for i in cat_min_train.columns]
train = pd.merge(train,cat_min_train.reset_index(),on='client_id')

In [110]:
med_df_train = transactions_train.groupby(['client_id','small_group'])['amount_rur'].median()
med_df_train = med_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_med_train = med_df_train.fillna(0)
cat_med_train.columns=['small_group_median_'+str(i) for i in cat_med_train.columns]
train = pd.merge(train,cat_med_train.reset_index(),on='client_id')

In [111]:
train.head()

,client_id,bins,sum,mean,std,min,max,count,median,small_group_0,...,small_group_median_192,small_group_median_193,small_group_median_195,small_group_median_196,small_group_median_197,small_group_median_198,small_group_median_199,small_group_median_200,small_group_median_202,small_group_median_203
0,24662,2,30254.011,34.774725,72.037354,0.074,1227.314,870,21.6995,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1046,0,42548.570,52.015367,106.540962,0.550,1210.506,818,20.1500,0.001222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34089,2,26842.816,34.325852,59.927450,0.043,782.641,782,16.9020,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34848,1,15773.126,16.160990,14.224936,0.043,109.590,976,12.0730,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,47076,3,12488.375,15.929050,35.473591,0.432,541.165,784,8.7065,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Проделаем аналогичное на тестовом датасете.

In [112]:
transactions_test = pd.read_csv('/home/alex/Рабочий стол/data_nti/data/transactions_test.csv')

test_id = pd.read_csv('/home/alex/Рабочий стол/data_nti/data/test.csv')

In [113]:
agg_features_test = transactions_test.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max', 'count', 'median']).reset_index()

In [114]:
test = pd.merge(test_id,agg_features_test,on='client_id')

In [115]:
counter_df_test = transactions_test.groupby(['client_id','small_group'])['amount_rur'].count()
cat_counts_test = counter_df_test.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_counts_test = cat_counts_test.fillna(0)
cat_counts_test.columns = ['small_group_'+str(i) for i in cat_counts_test.columns]
test = pd.merge(test,cat_counts_test.reset_index(),on='client_id')
for name in cat_counts_test.columns:
    test[name] /= test['count']

In [116]:
summer_df_test = transactions_test.groupby(['client_id','small_group'])['amount_rur'].sum()
cat_sums_test = summer_df_test.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_sums_test = cat_sums_test.fillna(0)
cat_sums_test.columns=['small_group_sum_'+str(i) for i in cat_sums_test.columns]
test = pd.merge(test,cat_sums_test.reset_index(),on='client_id')
for name in cat_sums_test.columns:
    test[name] /= test['sum']

In [117]:
mean_df_test = transactions_test.groupby(['client_id','small_group'])['amount_rur'].mean()
mean_df_test = mean_df_test.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_mean_test = mean_df_test.fillna(0)
cat_mean_test.columns=['small_group_mean_'+str(i) for i in cat_mean_test.columns]
test = pd.merge(test,cat_mean_test.reset_index(),on='client_id')

In [118]:
std_df_test = transactions_test.groupby(['client_id','small_group'])['amount_rur'].std()
std_df_test = std_df_test.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_std_test = std_df_test.fillna(0)
cat_std_test.columns=['small_group_std_'+str(i) for i in cat_std_test.columns]
test = pd.merge(test,cat_std_test.reset_index(),on='client_id')

In [119]:
max_df_test = transactions_test.groupby(['client_id','small_group'])['amount_rur'].max()
max_df_test = max_df_test.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_max_test = max_df_test.fillna(0)
cat_max_test.columns=['small_group_max_'+str(i) for i in cat_max_test.columns]
test = pd.merge(test,cat_max_test.reset_index(),on='client_id')

In [120]:
min_df_test = transactions_test.groupby(['client_id','small_group'])['amount_rur'].min()
min_df_test = min_df_test.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_min_test = min_df_test.fillna(0)
cat_min_test.columns=['small_group_min_'+str(i) for i in cat_min_test.columns]
test = pd.merge(test,cat_min_test.reset_index(),on='client_id')

In [121]:
med_df_test = transactions_test.groupby(['client_id','small_group'])['amount_rur'].median()
med_df_test = med_df_test.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_med_test = med_df_test.fillna(0)
cat_med_test.columns=['small_group_med_'+str(i) for i in cat_med_test.columns]
test = pd.merge(test,cat_med_test.reset_index(),on='client_id')

In [122]:
test.head()

,client_id,sum,mean,std,min,max,count,median,small_group_0,small_group_1,...,small_group_med_192,small_group_med_193,small_group_med_194,small_group_med_195,small_group_med_196,small_group_med_197,small_group_med_198,small_group_med_200,small_group_med_201,small_group_med_202
0,28571,30507.083,42.488974,43.659666,0.078,306.882,718,28.0745,0.000000,0.387187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,27046,39378.016,38.988135,93.600961,0.043,1469.007,1010,17.0350,0.008911,0.191089,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13240,50211.127,53.302683,198.583630,1.078,3902.918,942,16.4635,0.000000,0.240977,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19974,45371.491,53.252924,273.597147,0.432,5865.551,852,18.8575,0.049296,0.357981,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10505,90883.628,81.950972,119.408720,0.043,1921.341,1109,51.6070,0.000000,0.465284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Теперь поработаем с датой. Посчитаем количество уникальных дат из предположения, что отсортированные уникальные значения между собой отличаются на сутки. Из этого же предположения мы можем разделить даты на дни недели, взяв от уникального значения остаток от деления на 7. Кроме этого, можно проводить еще много экспериментов на проверку периодичности во времени - например, брать синус от даты и придумывать для этих значений фичи. Как и для групп товаров, посчитаем простые фичи для дней недели.

In [123]:
agg_features.head()

,client_id,sum,mean,std,min,max,count,median
0,4,28404.121,39.450168,73.511624,0.043,1341.802,720,22.725
1,6,15720.739,21.535259,26.200397,0.045,315.781,730,13.898
2,7,53630.036,69.379089,253.261383,0.043,4505.971,773,27.477
3,10,34419.365,48.752642,63.191701,0.045,654.893,706,32.057
4,11,26789.404,32.991877,107.395139,0.388,2105.058,812,11.066


In [124]:
train_uniq_day = pd.DataFrame(transactions_train.groupby('client_id').trans_date.nunique())
train = pd.merge(train,train_uniq_day.reset_index(),on='client_id')
train = train.rename(columns = {'trans_date': 'uniq_date'})

In [125]:
test_uniq_day = pd.DataFrame(transactions_test.groupby('client_id').trans_date.nunique())
test = pd.merge(test,test_uniq_day.reset_index(),on='client_id')
test = test.rename(columns = {'trans_date': 'uniq_date'})

In [126]:
for i in range(7):
    name_column = "unq_date_{}".format(i)
    trans_date_i = transactions_train[transactions_train['trans_date'] % 7 == i]
    trans_date_i = trans_date_i.groupby('client_id').trans_date.count()
    train = pd.merge(train,trans_date_i.reset_index(),on='client_id')
    train = train.rename(columns = {'trans_date': name_column})
    train[name_column] /= train['uniq_date'] 

In [127]:
for i in range(7):
    name_column = "unq_date_{}".format(i)
    trans_date_i = transactions_test[transactions_test['trans_date'] % 7 == i]
    trans_date_i = trans_date_i.groupby('client_id').trans_date.count()
    test = pd.merge(test,trans_date_i.reset_index(),on='client_id')
    test = test.rename(columns = {'trans_date': name_column})
    test[name_column] /= test['uniq_date']

In [128]:
transactions_train_i = transactions_train.copy()
transactions_train_i['trans_date'] %= 7
transactions_train_i = transactions_train_i.groupby(['client_id', 'trans_date'], as_index=False).amount_rur.aggregate(['mean', 'std', 'median', 'sum']).reset_index()
for j in range(7):
    name_column_mean = "mean_{}".format(j)
    name_column_std = "std_{}".format(j)
    name_column_median = "median_{}".format(j)
    name_column_sum = "sum_{}".format(j)
    transactions_train_i_j = transactions_train_i.copy().rename(columns={'mean': name_column_mean, 'std': name_column_std,
                                                                        'median': name_column_median, 'sum': name_column_sum})
    transactions_train_i_j = transactions_train_i_j[transactions_train_i_j['trans_date'] == j]
    transactions_train_i_j = transactions_train_i_j.drop('trans_date', axis=1)
    train = pd.merge(train, transactions_train_i_j, on='client_id')
    train[name_column_sum] /= train['sum']

In [129]:
transactions_test_i = transactions_test.copy()
transactions_test_i['trans_date'] %= 7
transactions_test_i = transactions_test_i.groupby(['client_id', 'trans_date'], as_index=False).amount_rur.aggregate(['mean', 'std', 'median', 'sum']).reset_index()
for j in range(7):
    name_column_mean = "mean_{}".format(j)
    name_column_std = "std_{}".format(j)
    name_column_median = "median_{}".format(j)
    name_column_sum = "sum_{}".format(j)
    transactions_test_i_j = transactions_test_i.copy().rename(columns={'mean': name_column_mean, 'std': name_column_std,
                                                                        'median': name_column_median, 'sum': name_column_sum})
    transactions_test_i_j = transactions_test_i_j[transactions_test_i_j['trans_date'] == j]
    transactions_test_i_j = transactions_test_i_j.drop('trans_date', axis=1)
    test = pd.merge(test, transactions_test_i_j, on='client_id')
    test[name_column_sum] /= test['sum']

In [130]:
train.head()

,client_id,bins,sum,mean,std,min,max,count,median,small_group_0,...,median_4,sum_4,mean_5,std_5,median_5,sum_5,mean_6,std_6,median_6,sum_6
0,24662,2,30254.011,34.774725,72.037354,0.074,1227.314,870,21.6995,0.000000,...,17.488,0.132299,41.655345,96.440358,22.8980,0.199644,40.047132,83.368687,24.867,0.201202
1,1046,0,42548.570,52.015367,106.540962,0.550,1210.506,818,20.1500,0.001222,...,13.028,0.091850,38.329283,56.368826,16.4860,0.089183,66.240520,108.263982,33.568,0.278671
2,34089,2,26842.816,34.325852,59.927450,0.043,782.641,782,16.9020,0.000000,...,20.062,0.155991,33.973000,43.838818,21.5240,0.153141,27.684984,40.359701,15.983,0.130985
3,34848,1,15773.126,16.160990,14.224936,0.043,109.590,976,12.0730,0.000000,...,13.006,0.158903,16.482868,14.004819,12.3650,0.157795,17.268631,16.269944,12.119,0.154369
4,47076,3,12488.375,15.929050,35.473591,0.432,541.165,784,8.7065,0.000000,...,8.013,0.139701,14.651133,32.247678,8.5495,0.140782,26.191703,71.164530,9.279,0.247480


In [131]:
test.head()

,client_id,sum,mean,std,min,max,count,median,small_group_0,small_group_1,...,median_4,sum_4,mean_5,std_5,median_5,sum_5,mean_6,std_6,median_6,sum_6
0,28571,30507.083,42.488974,43.659666,0.078,306.882,718,28.0745,0.000000,0.387187,...,22.806,0.116711,47.582090,43.648945,35.2160,0.190284,49.178548,54.080337,32.4750,0.185384
1,27046,39378.016,38.988135,93.600961,0.043,1469.007,1010,17.0350,0.008911,0.191089,...,17.310,0.218475,35.597043,86.368773,15.3330,0.146445,47.799692,75.076401,25.4790,0.173583
2,13240,50211.127,53.302683,198.583630,1.078,3902.918,942,16.4635,0.000000,0.240977,...,13.305,0.083001,54.433444,145.178300,17.0590,0.173455,58.076729,81.808474,31.0745,0.136485
3,19974,45371.491,53.252924,273.597147,0.432,5865.551,852,18.8575,0.049296,0.357981,...,18.547,0.085495,33.908598,48.790058,20.2830,0.094914,50.545328,70.752891,24.7230,0.145938
4,10505,90883.628,81.950972,119.408720,0.043,1921.341,1109,51.6070,0.000000,0.465284,...,56.274,0.159660,83.529045,166.442741,47.8285,0.143376,130.445321,164.996395,67.5595,0.232519


Посчитаем разброс как отдельную фичу

In [132]:
train['range_value'] = train['max'] - train['min']
replace = ['sum', 'min', 'max']
for name in replace:
    train[name + '_value'] = train[name].copy()
    del train[name]

In [133]:
test['range_value'] = test['max'] - test['min']
replace = ['sum', 'min', 'max']
for name in replace:
    test[name + '_value'] = test[name].copy()
    del test[name]

Разобьем данные на обучающую и валидационную выборки

In [134]:
X = train.copy()
y = X['bins']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = .2, random_state = 53)

Посмотрим на агрегацию по отдельным возрастным группам некоторых из фичей. Мы видим, что в некоторых из них среднеквадратическое отклонение сильно превосходит среднее значение. Из этого можно предположить, что в наших данных есть выбросы и нужно сделать фильтрацию, но только в обучающей выборке, в валидации фильтрацию делать не нужно.

In [136]:
aggregable_features = ['sum_value', 'mean', 'std', 'max_value', 'range_value']
classes = sorted(train['bins'].unique())
X_train.groupby('bins')[aggregable_features].agg(['mean', 'std'])

sum_value                     mean                     std  \
              mean           std       mean         std        mean   
bins                                                                  
0     46017.759420  38987.757597  51.534932   41.817186  140.437801   
1     61840.227698  98240.670248  71.421559  118.507099  194.912186   
2     28348.564069  19714.152045  31.835603   22.216063   84.605634   
3     60225.991698  61948.460005  67.859980   68.318365  184.323455   

                    max_value               range_value               
             std         mean          std         mean          std  
bins                                                                  
0     204.956750  2622.331520  5081.198589  2622.031301  5081.181466  
1     525.171377  3512.530950  9982.457975  3511.974880  9982.411729  
2     142.825290  1584.728033  3424.003768  1584.532289  3424.003309  
3     336.440313  3366.241610  7897.497458  3365.848563  7897.491179

In [137]:
percentile = {}
for i in classes:
    percentile[i] = {'conditions': '('}
    for feature in aggregable_features[:-1]:
        percentile[i][feature] = X_train[X_train['bins'] == i][feature].quantile(0.99)
        percentile[i]['conditions'] += '({} < {}) & '.format(feature, percentile[i][feature])
    percentile[i]['conditions'] = percentile[i]['conditions'][:-3] + ')'

In [138]:
for i in classes:
    string = '(bins != {}) | '.format(i) + percentile[i]['conditions']
    X_train = X_train.query(string)
y_train = X_train['bins']

In [139]:
aggregated = X_train.groupby('bins')[aggregable_features].agg(['mean', 'std'])
aggregated

sum_value                     mean                    std  \
              mean           std       mean        std        mean   
bins                                                                 
0     42858.346066  22424.987592  48.158434  24.263883  119.702177   
1     54839.663268  41826.021659  63.468489  46.474759  155.081508   
2     26832.384511  12086.089707  30.167940  12.964201   73.094631   
3     54898.150008  34699.001785  61.942036  38.034669  152.589752   

                    max_value               range_value               
             std         mean          std         mean          std  
bins                                                                  
0      99.974262  2143.636139  2362.799875  2143.339616  2362.785507  
1     165.569946  2674.023256  3424.665212  2673.472337  3424.635950  
2      50.812525  1315.639091  1181.001689  1315.444306  1180.993778  
3     144.628946  2670.239065  3169.858273  2669.849869  3169.862624

Выделим common features для обучения пересечением фич в тестовой и тренировочной выборках. Также можно протестировать предположение о том, что фича, представленная пресказанной регрессионной моделью может улучшить итоговое качество.

In [140]:
common_features=list(set(X_train.columns).intersection(set(test.columns)))

In [141]:
common_features.remove('client_id')

In [142]:
# y_train, y_valid
X_train_common = X_train[common_features]
X_valid_common = X_valid[common_features]
X_test_common = test[common_features]

In [143]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(loss='huber')
sgd.fit(X_train_common, y_train)
X_train_common['classifier'] = sgd.predict(X_train_common)[None].T
X_valid_common['classifier'] = sgd.predict(X_valid_common)[None].T
X_test_common['classifier'] = sgd.predict(X_test_common)[None].T

/home/alex/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [144]:
X_train_common

,small_group_std_170,small_group_min_23,small_group_mean_20,small_group_std_63,small_group_mean_170,small_group_mean_143,small_group_sum_83,small_group_sum_130,small_group_max_80,small_group_14,...,small_group_103,small_group_sum_89,small_group_2,small_group_sum_98,small_group_min_145,small_group_11,small_group_max_27,small_group_mean_61,small_group_mean_109,classifier
2780,0.0,55.183,810.044571,15.857259,0.0,0.0,0.000000,0.0,215.238,0.001965,...,0.000000,0.00000,0.054028,0.000000,0.0,0.174853,0.000,131.673800,0.000,353156.245517
2568,0.0,80.920,39.028150,0.000000,0.0,0.0,0.000000,0.0,0.000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.0,0.003452,0.000,211.060833,0.000,293729.703212
18993,0.0,297.580,22.772500,0.000000,0.0,0.0,0.000000,0.0,0.000,0.000000,...,0.000000,0.00095,0.000000,0.000000,0.0,0.213909,0.000,0.000000,0.000,146138.063584
3409,0.0,155.259,524.509000,0.000000,0.0,0.0,0.000000,0.0,1367.173,0.003542,...,0.001181,0.00000,0.012987,0.000000,0.0,0.025974,0.000,15.524000,0.000,214355.490934
27589,0.0,467.066,284.574000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.000000,...,0.000000,0.00071,0.007455,0.000000,0.0,0.331203,0.000,0.000000,0.000,325683.757783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14075,0.0,16.820,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.009434,...,0.000000,0.00000,0.070755,0.000408,0.0,0.010613,0.000,0.000000,0.000,171479.975603
22949,0.0,274.314,444.814000,0.000000,0.0,0.0,0.000000,0.0,228.976,0.000000,...,0.000000,0.00000,0.002622,0.000000,0.0,0.098776,0.000,143.527778,0.000,260607.081033
2933,0.0,764.141,120.672400,0.000000,0.0,0.0,0.000000,0.0,0.000,0.001189,...,0.000000,0.00000,0.061831,0.000000,0.0,0.254459,135.096,0.000000,0.000,254923.685086
25373,0.0,12.938,91.667800,0.000000,0.0,0.0,0.001153,0.0,408.645,0.000000,...,0.000000,0.00000,0.004530,0.001490,0.0,0.091733,0.000,96.078000,0.000,175356.404326


In [145]:
common_features = list(set(X_train_common.columns))

CatBoost - наилучшая модель для работы с предсказанием категориальных данных. Обучим на данных, а затем очистим с помощью feature_importances от шума наши фичи и еще раз обучим на очищенных данных.

In [54]:
cat = CatBoostClassifier(iterations = 5000, learning_rate = 0.05, depth = 5, loss_function = 'MultiClass', eval_metric='Accuracy',
                         random_state = 53, use_best_model=True)

train_data = Pool(
    data = X_train_common[common_features],
    label = y_train
)

valid_data = Pool(
    data = X_valid_common[common_features],
    label = y_valid
)


In [55]:
cat.fit(train_data, eval_set = valid_data)

0:	learn: 0.5026720	test: 0.4958333	best: 0.4958333 (0)	total: 1.42s	remaining: 1h 58m 3s
1:	learn: 0.5234965	test: 0.5130000	best: 0.5130000 (1)	total: 2.65s	remaining: 1h 50m 20s
2:	learn: 0.5287556	test: 0.5231667	best: 0.5231667 (2)	total: 3.86s	remaining: 1h 47m 13s
3:	learn: 0.5381288	test: 0.5350000	best: 0.5350000 (3)	total: 5.03s	remaining: 1h 44m 40s
4:	learn: 0.5391891	test: 0.5381667	best: 0.5381667 (4)	total: 6.18s	remaining: 1h 42m 59s
5:	learn: 0.5447451	test: 0.5403333	best: 0.5403333 (5)	total: 7.33s	remaining: 1h 41m 42s
6:	learn: 0.5466961	test: 0.5438333	best: 0.5438333 (6)	total: 8.49s	remaining: 1h 40m 58s
7:	learn: 0.5499194	test: 0.5475000	best: 0.5475000 (7)	total: 9.65s	remaining: 1h 40m 21s
8:	learn: 0.5503860	test: 0.5450000	best: 0.5475000 (7)	total: 10.9s	remaining: 1h 40m 48s
9:	learn: 0.5507253	test: 0.5490000	best: 0.5490000 (9)	total: 12.1s	remaining: 1h 40m 41s
10:	learn: 0.5531003	test: 0.5451667	best: 0.5490000 (9)	total: 13.3s	remaining: 1h 40m 25s

89:	learn: 0.6084061	test: 0.5960000	best: 0.5960000 (85)	total: 1m 41s	remaining: 1h 32m 14s
90:	learn: 0.6089151	test: 0.5953333	best: 0.5960000 (85)	total: 1m 42s	remaining: 1h 32m 12s
91:	learn: 0.6092968	test: 0.5968333	best: 0.5968333 (91)	total: 1m 43s	remaining: 1h 32m 5s
92:	learn: 0.6099330	test: 0.5958333	best: 0.5968333 (91)	total: 1m 44s	remaining: 1h 31m 59s
93:	learn: 0.6100178	test: 0.5968333	best: 0.5968333 (91)	total: 1m 45s	remaining: 1h 31m 56s
94:	learn: 0.6107388	test: 0.5973333	best: 0.5973333 (94)	total: 1m 46s	remaining: 1h 31m 51s
95:	learn: 0.6110357	test: 0.5986667	best: 0.5986667 (95)	total: 1m 47s	remaining: 1h 31m 47s
96:	learn: 0.6109085	test: 0.5988333	best: 0.5988333 (96)	total: 1m 48s	remaining: 1h 31m 42s
97:	learn: 0.6108661	test: 0.5993333	best: 0.5993333 (97)	total: 1m 49s	remaining: 1h 31m 36s
98:	learn: 0.6107388	test: 0.5986667	best: 0.5993333 (97)	total: 1m 50s	remaining: 1h 31m 34s
99:	learn: 0.6113750	test: 0.5995000	best: 0.5995000 (99)	tot

176:	learn: 0.6341505	test: 0.6130000	best: 0.6130000 (176)	total: 3m 27s	remaining: 1h 34m 6s
177:	learn: 0.6349139	test: 0.6128333	best: 0.6130000 (176)	total: 3m 28s	remaining: 1h 34m 2s
178:	learn: 0.6347018	test: 0.6133333	best: 0.6133333 (178)	total: 3m 29s	remaining: 1h 33m 56s
179:	learn: 0.6346170	test: 0.6135000	best: 0.6135000 (179)	total: 3m 30s	remaining: 1h 33m 53s
180:	learn: 0.6352956	test: 0.6135000	best: 0.6135000 (179)	total: 3m 31s	remaining: 1h 33m 50s
181:	learn: 0.6358470	test: 0.6130000	best: 0.6135000 (179)	total: 3m 32s	remaining: 1h 33m 47s
182:	learn: 0.6361439	test: 0.6131667	best: 0.6135000 (179)	total: 3m 33s	remaining: 1h 33m 44s
183:	learn: 0.6361439	test: 0.6140000	best: 0.6140000 (183)	total: 3m 34s	remaining: 1h 33m 44s
184:	learn: 0.6361439	test: 0.6146667	best: 0.6146667 (184)	total: 3m 35s	remaining: 1h 33m 40s
185:	learn: 0.6365256	test: 0.6140000	best: 0.6146667 (184)	total: 3m 37s	remaining: 1h 33m 36s
186:	learn: 0.6366104	test: 0.6143333	best

262:	learn: 0.6537026	test: 0.6261667	best: 0.6266667 (254)	total: 5m 5s	remaining: 1h 31m 35s
263:	learn: 0.6537026	test: 0.6248333	best: 0.6266667 (254)	total: 5m 6s	remaining: 1h 31m 32s
264:	learn: 0.6541691	test: 0.6250000	best: 0.6266667 (254)	total: 5m 7s	remaining: 1h 31m 30s
265:	learn: 0.6541691	test: 0.6250000	best: 0.6266667 (254)	total: 5m 8s	remaining: 1h 31m 27s
266:	learn: 0.6548477	test: 0.6246667	best: 0.6266667 (254)	total: 5m 9s	remaining: 1h 31m 26s
267:	learn: 0.6552719	test: 0.6256667	best: 0.6266667 (254)	total: 5m 10s	remaining: 1h 31m 22s
268:	learn: 0.6549326	test: 0.6248333	best: 0.6266667 (254)	total: 5m 11s	remaining: 1h 31m 20s
269:	learn: 0.6551870	test: 0.6246667	best: 0.6266667 (254)	total: 5m 12s	remaining: 1h 31m 17s
270:	learn: 0.6553991	test: 0.6245000	best: 0.6266667 (254)	total: 5m 13s	remaining: 1h 31m 14s
271:	learn: 0.6556112	test: 0.6250000	best: 0.6266667 (254)	total: 5m 14s	remaining: 1h 31m 12s
272:	learn: 0.6561201	test: 0.6251667	best: 0

348:	learn: 0.6679532	test: 0.6306667	best: 0.6315000 (329)	total: 6m 55s	remaining: 1h 32m 17s
349:	learn: 0.6681228	test: 0.6301667	best: 0.6315000 (329)	total: 6m 56s	remaining: 1h 32m 17s
350:	learn: 0.6680380	test: 0.6303333	best: 0.6315000 (329)	total: 6m 57s	remaining: 1h 32m 13s
351:	learn: 0.6687590	test: 0.6308333	best: 0.6315000 (329)	total: 6m 58s	remaining: 1h 32m 9s
352:	learn: 0.6685470	test: 0.6305000	best: 0.6315000 (329)	total: 6m 59s	remaining: 1h 32m 8s
353:	learn: 0.6687166	test: 0.6301667	best: 0.6315000 (329)	total: 7m 1s	remaining: 1h 32m 5s
354:	learn: 0.6684197	test: 0.6305000	best: 0.6315000 (329)	total: 7m 2s	remaining: 1h 32m 2s
355:	learn: 0.6685894	test: 0.6305000	best: 0.6315000 (329)	total: 7m 3s	remaining: 1h 32m
356:	learn: 0.6691407	test: 0.6301667	best: 0.6315000 (329)	total: 7m 4s	remaining: 1h 31m 58s
357:	learn: 0.6692255	test: 0.6298333	best: 0.6315000 (329)	total: 7m 5s	remaining: 1h 31m 54s
358:	learn: 0.6699466	test: 0.6303333	best: 0.6315000

...

4606:	learn: 0.8929086	test: 0.6480000	best: 0.6485000 (3874)	total: 1h 20m 59s	remaining: 6m 54s
4607:	learn: 0.8929935	test: 0.6478333	best: 0.6485000 (3874)	total: 1h 21m 1s	remaining: 6m 53s
4608:	learn: 0.8930359	test: 0.6478333	best: 0.6485000 (3874)	total: 1h 21m 2s	remaining: 6m 52s
4609:	learn: 0.8929511	test: 0.6473333	best: 0.6485000 (3874)	total: 1h 21m 3s	remaining: 6m 51s
4610:	learn: 0.8929511	test: 0.6476667	best: 0.6485000 (3874)	total: 1h 21m 4s	remaining: 6m 50s
4611:	learn: 0.8930359	test: 0.6478333	best: 0.6485000 (3874)	total: 1h 21m 5s	remaining: 6m 49s
4612:	learn: 0.8929511	test: 0.6478333	best: 0.6485000 (3874)	total: 1h 21m 6s	remaining: 6m 48s
4613:	learn: 0.8930783	test: 0.6475000	best: 0.6485000 (3874)	total: 1h 21m 7s	remaining: 6m 47s
4614:	learn: 0.8929935	test: 0.6480000	best: 0.6485000 (3874)	total: 1h 21m 8s	remaining: 6m 46s
4615:	learn: 0.8931631	test: 0.6480000	best: 0.6485000 (3874)	total: 1h 21m 9s	remaining: 6m 45s
4616:	learn: 0.8930783	test: 

4690:	learn: 0.8953686	test: 0.6470000	best: 0.6488333 (4643)	total: 1h 22m 27s	remaining: 5m 25s
4691:	learn: 0.8954958	test: 0.6471667	best: 0.6488333 (4643)	total: 1h 22m 28s	remaining: 5m 24s
4692:	learn: 0.8957503	test: 0.6470000	best: 0.6488333 (4643)	total: 1h 22m 29s	remaining: 5m 23s
4693:	learn: 0.8956655	test: 0.6466667	best: 0.6488333 (4643)	total: 1h 22m 29s	remaining: 5m 22s
4694:	learn: 0.8957927	test: 0.6463333	best: 0.6488333 (4643)	total: 1h 22m 31s	remaining: 5m 21s
4695:	learn: 0.8957503	test: 0.6465000	best: 0.6488333 (4643)	total: 1h 22m 32s	remaining: 5m 20s
4696:	learn: 0.8957503	test: 0.6468333	best: 0.6488333 (4643)	total: 1h 22m 33s	remaining: 5m 19s
4697:	learn: 0.8957503	test: 0.6470000	best: 0.6488333 (4643)	total: 1h 22m 34s	remaining: 5m 18s
4698:	learn: 0.8956230	test: 0.6475000	best: 0.6488333 (4643)	total: 1h 22m 35s	remaining: 5m 17s
4699:	learn: 0.8956655	test: 0.6475000	best: 0.6488333 (4643)	total: 1h 22m 36s	remaining: 5m 16s
4700:	learn: 0.89566

4774:	learn: 0.8979557	test: 0.6460000	best: 0.6488333 (4643)	total: 1h 23m 55s	remaining: 3m 57s
4775:	learn: 0.8978709	test: 0.6458333	best: 0.6488333 (4643)	total: 1h 23m 56s	remaining: 3m 56s
4776:	learn: 0.8979981	test: 0.6456667	best: 0.6488333 (4643)	total: 1h 23m 57s	remaining: 3m 55s
4777:	learn: 0.8980405	test: 0.6460000	best: 0.6488333 (4643)	total: 1h 23m 58s	remaining: 3m 54s
4778:	learn: 0.8980405	test: 0.6458333	best: 0.6488333 (4643)	total: 1h 23m 59s	remaining: 3m 53s
4779:	learn: 0.8980830	test: 0.6456667	best: 0.6488333 (4643)	total: 1h 24m	remaining: 3m 51s
4780:	learn: 0.8982102	test: 0.6455000	best: 0.6488333 (4643)	total: 1h 24m 1s	remaining: 3m 50s
4781:	learn: 0.8982526	test: 0.6453333	best: 0.6488333 (4643)	total: 1h 24m 2s	remaining: 3m 49s
4782:	learn: 0.8982526	test: 0.6458333	best: 0.6488333 (4643)	total: 1h 24m 3s	remaining: 3m 48s
4783:	learn: 0.8981678	test: 0.6456667	best: 0.6488333 (4643)	total: 1h 24m 4s	remaining: 3m 47s
4784:	learn: 0.8982526	test:

4858:	learn: 0.9001188	test: 0.6461667	best: 0.6488333 (4643)	total: 1h 25m 23s	remaining: 2m 28s
4859:	learn: 0.9001188	test: 0.6465000	best: 0.6488333 (4643)	total: 1h 25m 24s	remaining: 2m 27s
4860:	learn: 0.9000763	test: 0.6461667	best: 0.6488333 (4643)	total: 1h 25m 25s	remaining: 2m 26s
4861:	learn: 0.8999491	test: 0.6456667	best: 0.6488333 (4643)	total: 1h 25m 26s	remaining: 2m 25s
4862:	learn: 0.9001188	test: 0.6456667	best: 0.6488333 (4643)	total: 1h 25m 27s	remaining: 2m 24s
4863:	learn: 0.9002884	test: 0.6460000	best: 0.6488333 (4643)	total: 1h 25m 28s	remaining: 2m 23s
4864:	learn: 0.9001188	test: 0.6458333	best: 0.6488333 (4643)	total: 1h 25m 29s	remaining: 2m 22s
4865:	learn: 0.9001188	test: 0.6458333	best: 0.6488333 (4643)	total: 1h 25m 30s	remaining: 2m 21s
4866:	learn: 0.9000763	test: 0.6458333	best: 0.6488333 (4643)	total: 1h 25m 31s	remaining: 2m 20s
4867:	learn: 0.9003308	test: 0.6451667	best: 0.6488333 (4643)	total: 1h 25m 32s	remaining: 2m 19s
4868:	learn: 0.90028

4943:	learn: 0.9020273	test: 0.6448333	best: 0.6488333 (4643)	total: 1h 27m 11s	remaining: 59.3s
4944:	learn: 0.9020697	test: 0.6450000	best: 0.6488333 (4643)	total: 1h 27m 12s	remaining: 58.2s
4945:	learn: 0.9020273	test: 0.6450000	best: 0.6488333 (4643)	total: 1h 27m 14s	remaining: 57.1s
4946:	learn: 0.9020697	test: 0.6448333	best: 0.6488333 (4643)	total: 1h 27m 15s	remaining: 56.1s
4947:	learn: 0.9020697	test: 0.6446667	best: 0.6488333 (4643)	total: 1h 27m 17s	remaining: 55s
4948:	learn: 0.9020273	test: 0.6445000	best: 0.6488333 (4643)	total: 1h 27m 18s	remaining: 54s
4949:	learn: 0.9018577	test: 0.6443333	best: 0.6488333 (4643)	total: 1h 27m 20s	remaining: 52.9s
4950:	learn: 0.9019001	test: 0.6446667	best: 0.6488333 (4643)	total: 1h 27m 21s	remaining: 51.9s
4951:	learn: 0.9019001	test: 0.6450000	best: 0.6488333 (4643)	total: 1h 27m 22s	remaining: 50.8s
4952:	learn: 0.9019849	test: 0.6448333	best: 0.6488333 (4643)	total: 1h 27m 24s	remaining: 49.8s
4953:	learn: 0.9021121	test: 0.645

In [56]:
feature_imp = pd.Series(cat.feature_importances_,index=common_features).sort_values(ascending=False)

In [68]:
feature_imp.head(60)

small_group_sum_11     5.154572
small_group_mean_1     3.066012
median_6               2.558598
small_group_12         1.558744
small_group_min_3      1.049154
small_group_sum_57     1.045631
median                 0.967782
small_group_sum_1      0.966566
small_group_std_12     0.912402
small_group_16         0.835259
small_group_25         0.803508
median_0               0.761656
small_group_61         0.754887
small_group_sum_3      0.730845
small_group_4          0.714173
small_group_sum_25     0.712360
small_group_36         0.698466
small_group_sum_36     0.675660
small_group_sum_61     0.673133
small_group_mean_25    0.639581
small_group_sum_106    0.628903
small_group_sum_12     0.604091
small_group_sum_24     0.600272
small_group_sum_18     0.595961
small_group_mean_4     0.575851
small_group_57         0.563987
uniq_date              0.551030
small_group_31         0.548064
small_group_min_25     0.545962
small_group_std_25     0.545570
small_group_1          0.537117
small_gr

In [58]:
feature_imp_idx = feature_imp.index.to_list()
feature_imp_idx

['small_group_sum_11',
 'small_group_mean_1',
 'median_6',
 'small_group_12',
 'small_group_min_3',
 'small_group_sum_57',
 'median',
 'small_group_sum_1',
 'small_group_std_12',
 'small_group_16',
 'small_group_25',
 'median_0',
 'small_group_61',
 'small_group_sum_3',
 'small_group_4',
 'small_group_sum_25',
 'small_group_36',
 'small_group_sum_36',
 'small_group_sum_61',
 'small_group_mean_25',
 'small_group_sum_106',
 'small_group_sum_12',
 'small_group_sum_24',
 'small_group_sum_18',
 'small_group_mean_4',
 'small_group_57',
 'uniq_date',
 'small_group_31',
 'small_group_min_25',
 'small_group_std_25',
 'small_group_1',
 'small_group_min_1',
 'small_group_sum_4',
 'small_group_mean_10',
 'small_group_11',
 'small_group_3',
 'small_group_mean_15',
 'small_group_24',
 'small_group_sum_31',
 'small_group_min_11',
 'median_5',
 'small_group_min_31',
 'small_group_std_57',
 'small_group_mean_11',
 'small_group_min_12',
 'median_3',
 'small_group_mean_61',
 'unq_date_0',
 'small_group_s

In [72]:
X_train_top = X_train_common[feature_imp_idx[:1000]]
X_valid_top = X_valid_common[feature_imp_idx[:1000]]
X_test_top = X_test_common[feature_imp_idx[:1000]]

train_data = Pool(
    data = X_train_top,
    label = y_train
)
valid_data = Pool(
    data = X_valid_top,
    label = y_valid
)

In [73]:
cat = CatBoostClassifier(iterations = 5000, learning_rate = 0.05, depth = 5, loss_function = 'MultiClass', 
                         eval_metric='Accuracy', random_state = 53, use_best_model=True)

cat.fit(train_data, eval_set = valid_data)

0:	learn: 0.5048774	test: 0.4958333	best: 0.4958333 (0)	total: 1.44s	remaining: 2h 24m 4s
1:	learn: 0.5201459	test: 0.5196667	best: 0.5196667 (1)	total: 2.65s	remaining: 2h 12m 22s
2:	learn: 0.5391891	test: 0.5353333	best: 0.5353333 (2)	total: 3.87s	remaining: 2h 8m 50s
3:	learn: 0.5441513	test: 0.5351667	best: 0.5353333 (2)	total: 5.11s	remaining: 2h 7m 36s
4:	learn: 0.5474595	test: 0.5446667	best: 0.5446667 (4)	total: 6.27s	remaining: 2h 5m 18s
5:	learn: 0.5519552	test: 0.5441667	best: 0.5446667 (4)	total: 7.64s	remaining: 2h 7m 9s
6:	learn: 0.5522945	test: 0.5440000	best: 0.5446667 (4)	total: 8.84s	remaining: 2h 6m 12s
7:	learn: 0.5522097	test: 0.5435000	best: 0.5446667 (4)	total: 10s	remaining: 2h 4m 48s
8:	learn: 0.5558572	test: 0.5470000	best: 0.5470000 (8)	total: 11.1s	remaining: 2h 3m 11s
9:	learn: 0.5566630	test: 0.5506667	best: 0.5506667 (9)	total: 12.2s	remaining: 2h 1m 58s
10:	learn: 0.5555603	test: 0.5523333	best: 0.5523333 (10)	total: 13.3s	remaining: 2h 54s
11:	learn: 0.

89:	learn: 0.6094665	test: 0.5963333	best: 0.5970000 (85)	total: 1m 48s	remaining: 1h 59m 9s
90:	learn: 0.6100178	test: 0.5981667	best: 0.5981667 (90)	total: 1m 49s	remaining: 1h 59m 1s
91:	learn: 0.6104419	test: 0.5998333	best: 0.5998333 (91)	total: 1m 51s	remaining: 1h 58m 48s
92:	learn: 0.6110357	test: 0.6008333	best: 0.6008333 (92)	total: 1m 52s	remaining: 1h 58m 45s
93:	learn: 0.6117143	test: 0.6015000	best: 0.6015000 (93)	total: 1m 53s	remaining: 1h 59m 1s
94:	learn: 0.6120112	test: 0.6028333	best: 0.6028333 (94)	total: 1m 55s	remaining: 1h 59m 26s
95:	learn: 0.6129019	test: 0.6016667	best: 0.6028333 (94)	total: 1m 56s	remaining: 1h 59m 13s
96:	learn: 0.6128170	test: 0.6021667	best: 0.6028333 (94)	total: 1m 57s	remaining: 1h 59m 5s
97:	learn: 0.6134532	test: 0.6010000	best: 0.6028333 (94)	total: 1m 58s	remaining: 1h 58m 54s
98:	learn: 0.6137077	test: 0.6028333	best: 0.6028333 (94)	total: 1m 59s	remaining: 1h 58m 46s
99:	learn: 0.6145984	test: 0.6020000	best: 0.6028333 (94)	total:

176:	learn: 0.6334295	test: 0.6138333	best: 0.6143333 (175)	total: 3m 44s	remaining: 2h 3m 20s
177:	learn: 0.6340657	test: 0.6135000	best: 0.6143333 (175)	total: 3m 46s	remaining: 2h 3m 35s
178:	learn: 0.6340657	test: 0.6140000	best: 0.6143333 (175)	total: 3m 48s	remaining: 2h 3m 44s
179:	learn: 0.6347018	test: 0.6131667	best: 0.6143333 (175)	total: 3m 50s	remaining: 2h 4m 5s
180:	learn: 0.6349563	test: 0.6133333	best: 0.6143333 (175)	total: 3m 52s	remaining: 2h 4m 31s
181:	learn: 0.6344898	test: 0.6136667	best: 0.6143333 (175)	total: 3m 54s	remaining: 2h 4m 59s
182:	learn: 0.6348291	test: 0.6136667	best: 0.6143333 (175)	total: 3m 57s	remaining: 2h 5m 52s
183:	learn: 0.6351684	test: 0.6138333	best: 0.6143333 (175)	total: 3m 59s	remaining: 2h 6m 11s
184:	learn: 0.6354229	test: 0.6148333	best: 0.6148333 (184)	total: 4m 1s	remaining: 2h 6m 31s
185:	learn: 0.6354229	test: 0.6143333	best: 0.6148333 (184)	total: 4m 2s	remaining: 2h 6m 35s
186:	learn: 0.6356773	test: 0.6138333	best: 0.6148333

263:	learn: 0.6539571	test: 0.6258333	best: 0.6258333 (260)	total: 5m 38s	remaining: 2h 2m 34s
264:	learn: 0.6545509	test: 0.6255000	best: 0.6258333 (260)	total: 5m 39s	remaining: 2h 2m 30s
265:	learn: 0.6550598	test: 0.6250000	best: 0.6258333 (260)	total: 5m 40s	remaining: 2h 2m 27s
266:	learn: 0.6550174	test: 0.6251667	best: 0.6258333 (260)	total: 5m 41s	remaining: 2h 2m 21s
267:	learn: 0.6553143	test: 0.6240000	best: 0.6258333 (260)	total: 5m 42s	remaining: 2h 2m 12s
268:	learn: 0.6551446	test: 0.6248333	best: 0.6258333 (260)	total: 5m 43s	remaining: 2h 2m 8s
269:	learn: 0.6548053	test: 0.6246667	best: 0.6258333 (260)	total: 5m 45s	remaining: 2h 2m 2s
270:	learn: 0.6548477	test: 0.6253333	best: 0.6258333 (260)	total: 5m 46s	remaining: 2h 1m 55s
271:	learn: 0.6552295	test: 0.6255000	best: 0.6258333 (260)	total: 5m 47s	remaining: 2h 1m 48s
272:	learn: 0.6554839	test: 0.6256667	best: 0.6258333 (260)	total: 5m 48s	remaining: 2h 1m 40s
273:	learn: 0.6553567	test: 0.6261667	best: 0.626166

350:	learn: 0.6710493	test: 0.6283333	best: 0.6301667 (317)	total: 7m 15s	remaining: 1h 56m 50s
351:	learn: 0.6711341	test: 0.6288333	best: 0.6301667 (317)	total: 7m 17s	remaining: 1h 56m 57s
352:	learn: 0.6711341	test: 0.6288333	best: 0.6301667 (317)	total: 7m 18s	remaining: 1h 56m 51s
353:	learn: 0.6713038	test: 0.6293333	best: 0.6301667 (317)	total: 7m 19s	remaining: 1h 56m 49s
354:	learn: 0.6708372	test: 0.6290000	best: 0.6301667 (317)	total: 7m 20s	remaining: 1h 56m 43s
355:	learn: 0.6709220	test: 0.6295000	best: 0.6301667 (317)	total: 7m 21s	remaining: 1h 56m 38s
356:	learn: 0.6708372	test: 0.6295000	best: 0.6301667 (317)	total: 7m 22s	remaining: 1h 56m 34s
357:	learn: 0.6711341	test: 0.6290000	best: 0.6301667 (317)	total: 7m 23s	remaining: 1h 56m 28s
358:	learn: 0.6714310	test: 0.6286667	best: 0.6301667 (317)	total: 7m 24s	remaining: 1h 56m 24s
359:	learn: 0.6715158	test: 0.6288333	best: 0.6301667 (317)	total: 7m 25s	remaining: 1h 56m 20s
360:	learn: 0.6713462	test: 0.6286667	be

...

5602:	learn: 0.9242938	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 33m 8s	remaining: 6m 35s
5603:	learn: 0.9243362	test: 0.6448333	best: 0.6473333 (1480)	total: 1h 33m 9s	remaining: 6m 34s
5604:	learn: 0.9243787	test: 0.6450000	best: 0.6473333 (1480)	total: 1h 33m 10s	remaining: 6m 33s
5605:	learn: 0.9243787	test: 0.6448333	best: 0.6473333 (1480)	total: 1h 33m 11s	remaining: 6m 32s
5606:	learn: 0.9243787	test: 0.6448333	best: 0.6473333 (1480)	total: 1h 33m 12s	remaining: 6m 31s
5607:	learn: 0.9243362	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 33m 13s	remaining: 6m 30s
5608:	learn: 0.9242090	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 33m 14s	remaining: 6m 29s
5609:	learn: 0.9242514	test: 0.6448333	best: 0.6473333 (1480)	total: 1h 33m 15s	remaining: 6m 28s
5610:	learn: 0.9242514	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 33m 16s	remaining: 6m 27s
5611:	learn: 0.9243362	test: 0.6451667	best: 0.6473333 (1480)	total: 1h 33m 17s	remaining: 6m 26s
5612:	learn: 0.9242938

5686:	learn: 0.9256510	test: 0.6440000	best: 0.6473333 (1480)	total: 1h 34m 30s	remaining: 5m 12s
5687:	learn: 0.9256934	test: 0.6436667	best: 0.6473333 (1480)	total: 1h 34m 31s	remaining: 5m 11s
5688:	learn: 0.9258631	test: 0.6440000	best: 0.6473333 (1480)	total: 1h 34m 32s	remaining: 5m 10s
5689:	learn: 0.9257783	test: 0.6443333	best: 0.6473333 (1480)	total: 1h 34m 33s	remaining: 5m 9s
5690:	learn: 0.9259055	test: 0.6445000	best: 0.6473333 (1480)	total: 1h 34m 34s	remaining: 5m 8s
5691:	learn: 0.9257783	test: 0.6445000	best: 0.6473333 (1480)	total: 1h 34m 35s	remaining: 5m 7s
5692:	learn: 0.9257359	test: 0.6443333	best: 0.6473333 (1480)	total: 1h 34m 36s	remaining: 5m 6s
5693:	learn: 0.9257359	test: 0.6445000	best: 0.6473333 (1480)	total: 1h 34m 37s	remaining: 5m 5s
5694:	learn: 0.9259479	test: 0.6443333	best: 0.6473333 (1480)	total: 1h 34m 38s	remaining: 5m 4s
5695:	learn: 0.9259479	test: 0.6441667	best: 0.6473333 (1480)	total: 1h 34m 39s	remaining: 5m 3s
5696:	learn: 0.9259479	test

5770:	learn: 0.9281534	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 35m 53s	remaining: 3m 48s
5771:	learn: 0.9281110	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 35m 54s	remaining: 3m 47s
5772:	learn: 0.9281534	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 35m 55s	remaining: 3m 46s
5773:	learn: 0.9281534	test: 0.6443333	best: 0.6473333 (1480)	total: 1h 35m 56s	remaining: 3m 45s
5774:	learn: 0.9281958	test: 0.6443333	best: 0.6473333 (1480)	total: 1h 35m 57s	remaining: 3m 44s
5775:	learn: 0.9282382	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 35m 58s	remaining: 3m 43s
5776:	learn: 0.9282806	test: 0.6445000	best: 0.6473333 (1480)	total: 1h 35m 58s	remaining: 3m 42s
5777:	learn: 0.9282806	test: 0.6445000	best: 0.6473333 (1480)	total: 1h 35m 59s	remaining: 3m 41s
5778:	learn: 0.9282382	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 36m	remaining: 3m 40s
5779:	learn: 0.9283230	test: 0.6445000	best: 0.6473333 (1480)	total: 1h 36m 1s	remaining: 3m 39s
5780:	learn: 0.9284503	te

5854:	learn: 0.9292561	test: 0.6456667	best: 0.6473333 (1480)	total: 1h 37m 15s	remaining: 2m 24s
5855:	learn: 0.9293833	test: 0.6453333	best: 0.6473333 (1480)	total: 1h 37m 16s	remaining: 2m 23s
5856:	learn: 0.9294257	test: 0.6455000	best: 0.6473333 (1480)	total: 1h 37m 17s	remaining: 2m 22s
5857:	learn: 0.9295530	test: 0.6453333	best: 0.6473333 (1480)	total: 1h 37m 18s	remaining: 2m 21s
5858:	learn: 0.9295106	test: 0.6453333	best: 0.6473333 (1480)	total: 1h 37m 19s	remaining: 2m 20s
5859:	learn: 0.9294257	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 37m 20s	remaining: 2m 19s
5860:	learn: 0.9295530	test: 0.6445000	best: 0.6473333 (1480)	total: 1h 37m 21s	remaining: 2m 18s
5861:	learn: 0.9295530	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 37m 22s	remaining: 2m 17s
5862:	learn: 0.9295954	test: 0.6445000	best: 0.6473333 (1480)	total: 1h 37m 23s	remaining: 2m 16s
5863:	learn: 0.9296802	test: 0.6448333	best: 0.6473333 (1480)	total: 1h 37m 24s	remaining: 2m 15s
5864:	learn: 0.92972

5938:	learn: 0.9309102	test: 0.6453333	best: 0.6473333 (1480)	total: 1h 38m 37s	remaining: 1m
5939:	learn: 0.9309102	test: 0.6451667	best: 0.6473333 (1480)	total: 1h 38m 38s	remaining: 59.8s
5940:	learn: 0.9309102	test: 0.6450000	best: 0.6473333 (1480)	total: 1h 38m 39s	remaining: 58.8s
5941:	learn: 0.9311222	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 38m 40s	remaining: 57.8s
5942:	learn: 0.9311222	test: 0.6445000	best: 0.6473333 (1480)	total: 1h 38m 41s	remaining: 56.8s
5943:	learn: 0.9310798	test: 0.6445000	best: 0.6473333 (1480)	total: 1h 38m 42s	remaining: 55.8s
5944:	learn: 0.9312495	test: 0.6448333	best: 0.6473333 (1480)	total: 1h 38m 43s	remaining: 54.8s
5945:	learn: 0.9312919	test: 0.6448333	best: 0.6473333 (1480)	total: 1h 38m 44s	remaining: 53.8s
5946:	learn: 0.9313343	test: 0.6446667	best: 0.6473333 (1480)	total: 1h 38m 45s	remaining: 52.8s
5947:	learn: 0.9312071	test: 0.6448333	best: 0.6473333 (1480)	total: 1h 38m 46s	remaining: 51.8s
5948:	learn: 0.9312071	test: 0.64

Сделаем submit по предсказанию нашей модели

In [74]:
pred = cat.predict(X_test_common).reshape(-1)

In [75]:
pred = list(map(int, pred))

In [76]:
submission = pd.DataFrame({'bins': pred}, index=test.client_id)
submission.head()

,bins
client_id,
28571,0
27046,2
13240,3
19974,2
10505,1


In [77]:
import time
import os

current_timestamp = int(time.time())
submission_path = '/home/alex/Рабочий стол/data_nti/submission/{}.csv'.format(current_timestamp)

if not os.path.exists('/home/alex/Рабочий стол/data_nti/submission/'):
    os.makedirs('/home/alex/Рабочий стол/data_nti/submission/')

print(submission_path)
submission.to_csv(submission_path, index=True)

/home/alex/Рабочий стол/data_nti/submission/1578485621.csv
